In [2]:
import tensorflow as tf
import numpy as np
tf.reset_default_graph()

## Teach RNN 'hihllo'

RNN에게 hihello를 학습시키고 그 다음 hihell를 입력하여 ihello를 출력하는것이 목표다.

즉 h를 입력하면 다음문자인 i가 나오고 i를 입력하면 다음문자인 h가 나오게 하는것이다.

이것이 선형회귀같은 일반적인 forward net에서 못하는 이유는 같은 h를 넣었어도 뒤에 i나e가 나올 수 있기 때문이다.

우리가 가르칠 text는 'hihello'이다.

여기서 쓰인 문자를 중복없이 나열 하면 h, i e, l, o이다.

따라서 각각의 문자를 index 시켜준다.

     h:0    i:1    e:2   l:3    o:4
     
그 후 인덱스를 원핫인코딩 시켜준다.     

In [3]:
idx2char = ['h', 'i', 'e', 'l', 'o']

x_data = [[0, 1, 0, 2, 3, 3]]   # hihell
x_one_hot = [[[1, 0, 0, 0, 0],   # h 0
              [0, 1, 0, 0, 0],   # i 1
              [1, 0, 0, 0, 0],   # h 0
              [0, 0, 1, 0, 0],   # e 2
              [0, 0, 0, 1, 0],   # l 3
              [0, 0, 0, 1, 0]]]  # l 3

우리가 원하는 출력은 ihello 이므로[1, 0, 2, 3, 3, 4]을 출력해야한다.

In [4]:
y_data = [[1, 0, 2, 3, 3, 4]]

우리가 주는 x_data가 6글자 이므로 sequence_length는 6이다

hidden은 출력은 우리가 원하는대로 결정할 수 있는데 원핫인코딩을 쓰므로 5여야한다.

batch는 문자열이 하나이므로  1이다.

In [5]:
hidden_size=5 
input_dim=5
batch_size=1
sequence_length = 6

이제 RNN모델을 세워보자 

드라이브 함수로 출력을 정해주기 위해 입력의 사이즈를 정해야한다.

입력(X)은 원핫인코딩을 쓰고 6글자를 사용하기 때문에 (n,6,5)여야 한다.

Y는 Label로 표현되므로 반복하기때문에 (n,6)이 된다.

이때 n는 배치 사이즈이다

In [6]:
X=tf.placeholder(tf.float32,[None,sequence_length,input_dim])
Y=tf.placeholder(tf.int32, [None,sequence_length])

이제 cell을 만들어 보자.

이때 initial state를 0으로 주자.

# 왜 0으로 주는 걸까


In [7]:
cell = tf.contrib.rnn.BasicLSTMCell(num_units=hidden_size, state_is_tuple=True)
initial_state = cell.zero_state(batch_size, tf.float32)
outputs, _states = tf.nn.dynamic_rnn(cell, X, initial_state=initial_state, dtype=tf.float32)

output이 얼마나 좋은가 loss로 계산해 봐야 한다.

## cost: sequence_loss

sequence_loss가 어떻게 작동하는지 알아보기 위해 한가지 예제를 살펴보자

In [8]:
sample_data = tf.constant([[1,1,1]])

이렇게 y 데이터 주어질때 값을 원핫으로 예측했다고 해보자

[batch size, sequence_length, emb_dim]

In [9]:
predicton = tf.constant([[[0.2,0.7],[0.6,0.2],[0.2,0.9]]],dtype=tf.float32) # 1, 0, 1로 예측

weights= tf.constant([[1,1,1]],dtype=tf.float32) # 전부 1로 둠

이제 이 값을 sequence_loss에 대입

In [10]:
sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=predicton, targets=sample_data, weights=weights)
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

print("Loss:",sequence_loss.eval())

Loss: 0.596759


 예측이 두가지 인 경우도 살펴보자

In [11]:
predicton1 = tf.constant([[[0.3,0.7],[0.3,0.7],[0.3,0.7]]],dtype=tf.float32)
predicton2 = tf.constant([[[0.1,0.9],[0.1,0.9],[0.1,0.9]]],dtype=tf.float32)

sequence_loss1 = tf.contrib.seq2seq.sequence_loss(logits=predicton1, targets=sample_data, weights=weights)
sequence_loss2 = tf.contrib.seq2seq.sequence_loss(logits=predicton2, targets=sample_data, weights=weights)
sess = tf.InteractiveSession()
sess.run(tf.global_variables_initializer())

print("Loss:",sequence_loss1.eval(),"\n","Loss:",sequence_loss2.eval())

Loss: 0.513015 
 Loss: 0.371101


정확하게 맞출 수록 오차가 줄어든다.

이제 다시 우리 모델로 돌아와 보자

weight는 전부 1로 해주자

In [12]:
weights = tf.ones([batch_size,sequence_length])

sequence_loss = tf.contrib.seq2seq.sequence_loss(logits=outputs, targets=Y, weights=weights)

loss = tf.reduce_mean(sequence_loss)
train= tf.train.AdamOptimizer(learning_rate=0.1).minimize(loss)

위와 같이 해도 되지만 rnn에서 나온 값을 바로 logit에 넣으면 안된다. 이 부분에 대해서는 뒤에서 살펴보자

이제 학습 시켜보자

In [13]:
prediction = tf.argmax(outputs, axis=2) # 마지막 축 (aixs=2)에서 원핫 인코딩이 이루어지고, 그것중 가장 큰 값으로 예측했으므로 argmax로 뽑아온다.


with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    for i in range(50):
        l, _ = sess.run([loss, train], feed_dict={X: x_one_hot, Y: y_data})
        result = sess.run(prediction, feed_dict={X: x_one_hot})
        print(i, "loss:", l, "prediction: ", result, "true Y: ", y_data)

0 loss: 1.59796 prediction:  [[2 3 3 3 3 4]] true Y:  [[1, 0, 2, 3, 3, 4]]
1 loss: 1.50784 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
2 loss: 1.42344 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
3 loss: 1.36237 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
4 loss: 1.31126 prediction:  [[2 3 3 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
5 loss: 1.25841 prediction:  [[2 3 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
6 loss: 1.20628 prediction:  [[2 3 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
7 loss: 1.16079 prediction:  [[2 3 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
8 loss: 1.13189 prediction:  [[2 3 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
9 loss: 1.09818 prediction:  [[2 3 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
10 loss: 1.06947 prediction:  [[2 3 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
11 loss: 1.0464 prediction:  [[2 3 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
12 loss: 1.02468 prediction:  [[2 0 2 3 3 3]] true Y:  [[1, 0, 2, 3, 3, 4]]
13 loss: 1.00294 predic

결과를 출력해보자

In [14]:
result_str = [idx2char[c] for c in np.squeeze(result)]
print("\tPrediction str: ", ''.join(result_str))

	Prediction str:  ihello


## 참고사이트
1. https://www.youtube.com/watch?v=39_P23TqUnw&index=43&list=PLlMkM4tgfjnLSOjrEJN31gZATbcj_MpUm
2. https://github.com/hunkim/DeepLearningZeroToAll/blob/master/lab-12-1-hello-rnn.py